In [ ]:
import pandas as pd
import plotly.express as px
import os

# 1. Configurazione Percorsi e Dati Statici
path = "./datasets/consumo_medio.csv"

prezzi_pacco = {
    2000: 2.05, 2001: 2.15, 2002: 2.76, 2003: 3.12, 2004: 3.28,
    2005: 3.48, 2006: 3.65, 2007: 3.80, 2008: 3.86, 2009: 4.02,
    2010: 4.11, 2011: 4.25, 2012: 4.35, 2013: 4.45, 2014: 4.51,
    2015: 4.57, 2016: 4.68, 2017: 4.74, 2018: 4.79, 2019: 4.86,
    2020: 4.90, 2021: 4.93, 2022: 4.98, 2023: 5.12, 2024: 5.38,
    2025: 5.68
}

def generate_expenditure_chart():
    # Verifica esistenza file
    if not os.path.exists(path):
        print(f"ERRORE: Il file '{path}' non è stato trovato.")
        return

    try:
        # 2. Caricamento e Filtro Dati
        df = pd.read_csv(path)

        # Filtro: Italia e indicatore specifico
        df_it = df[
            (df["Territorio"] == "Italia") &
            (df["Indicatore"].str.contains("Numero medio di sigarette", na=False))
        ].copy()

        if df_it.empty:
            print("Nessun dato trovato per i filtri specificati.")
            return

        # Pulizia e ridenominazione
        consumo = df_it[["TIME_PERIOD", "Osservazione"]].copy()
        consumo.columns = ["Anno", "Sigarette_medie_giornaliere_per_100_persone"]
        consumo["Anno"] = pd.to_numeric(consumo["Anno"], errors='coerce')
        consumo = consumo.dropna(subset=["Anno"]).sort_values("Anno")

        # 3. Calcolo Spesa
        # Prezzo per sigaretta (Prezzo pacchetto / 20)
        consumo["Prezzo_sigaretta"] = consumo["Anno"].map(lambda x: prezzi_pacco.get(x, None))

        # Rimuoviamo anni per cui non abbiamo il prezzo (se necessario)
        consumo = consumo.dropna(subset=["Prezzo_sigaretta"])
        consumo["Prezzo_sigaretta"] = consumo["Prezzo_sigaretta"] / 20

        # Spesa media giornaliera per 100 persone
        consumo["Spesa_media_giornaliera_per_100"] = (
            consumo["Sigarette_medie_giornaliere_per_100_persone"] * consumo["Prezzo_sigaretta"]
        )

        # Arrotondamento per pulizia grafico
        consumo["Spesa_media_giornaliera_per_100"] = consumo["Spesa_media_giornaliera_per_100"].round(2)

        # 4. Creazione Grafico Plotly
        fig = px.line(
            consumo,
            x="Anno",
            y="Spesa_media_giornaliera_per_100",
            markers=True,
            labels={
                "Anno": "Anno",
                "Spesa_media_giornaliera_per_100": "Spesa (€)"
            },
            title="<b>Spesa media giornaliera per 100 persone (Sigarette)</b><br>Fonte: ISTAT ed ADM"
        )

        # 5. Styling (Layout identico al tuo esempio)
        fig.update_layout(
            font=dict(family="Arial", size=14),
            hovermode="x unified",
            template="plotly_white", # Sfondo pulito
            yaxis=dict(
                title="Spesa media giornaliera (€)",
                tickprefix="€", # Aggiunge il simbolo Euro all'asse Y
                showgrid=True
            ),
            xaxis=dict(
                showgrid=True
            ),
            margin=dict(r=50)
        )

        # Visualizzazione interattiva sui punti
        fig.update_traces(
            hovertemplate="<b>Anno: %{x}</b><br>Spesa: €%{y:.2f}<extra></extra>"
        )

        fig.show()

    except Exception as e:
        print(f"Errore durante l'elaborazione: {e}")

# Esecuzione
generate_expenditure_chart()